In [1]:
# !pip3 install -U openai-whisper -i https://mirrors.aliyun.com/pypi/simple/
# !pip3 install jiwer -i https://mirrors.aliyun.com/pypi/simple/
# !pip3 install noisereduce -i https://mirrors.aliyun.com/pypi/simple/ 

In [3]:
import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
from tqdm import tqdm
import torch
import librosa
import numpy as np
import glob
import os

import sys
sys.path.append("/home/shoinoue/Git/cuhksz-phd/sho_util/pyfiles/")
from sound import play_audio

import whisper
from whisper.normalizers.english import EnglishTextNormalizer
normalizer = EnglishTextNormalizer()
import jiwer
import pandas as pd
import scipy.stats as st
import math
from IPython.display import clear_output
from scipy.io.wavfile import write as write_wav

sr = 16000

## Whisper

In [4]:
files = glob.glob("./SPAT/transliteration/*.npy")
files.sort()
groundtruth_dir = {}
for file in files:
    key = os.path.basename(file)[:-4]
    text = np.load(file, allow_pickle=True).item()["Original English"]
    groundtruth_dir[key] = normalizer(text)

In [5]:
base_model_dir = "/mntcephfs/lab_data/shoinoue/Models/trained_models/AC_01/ckpts_16000/"
model_paths = [
    # "VTN_fine-tuning_nocondition_gt2syn_melmel_hubert_norepeating_small/100000",
    "VTN_fine-tuning_nocondition_gt2syn_melmel_hubert_norepeating/100000",
    # "VTN_fine-tuning_nocondition_mix2synVCTK3hr_melmel_hubert_norepeating_small/100000",
    "VTN_fine-tuning_nocondition_mix2synVCTK3hr_melmel_hubert_norepeating/100000",
    # "VTN_fine-tuning_nocondition_gt2syn_80mel80mel_hubert_norepeating_small/100000",
    "VTN_fine-tuning_nocondition_gt2syn_80mel80mel_hubert_norepeating/100000",
    # "VTN_fine-tuning_nocondition_mix2synVCTK3hr_80mel80mel_hubert_norepeating_small/100000",
    "VTN_fine-tuning_nocondition_mix2synVCTK3hr_80mel80mel_hubert_norepeating/100000",
    "VTN_fine-tuning_nocondition_gt2syn_hifiganmelhifiganmel_hubert_norepeating/100000",
    "VTN_fine-tuning_nocondition_mix2synVCTK3hr_hifiganmelhifiganmel_hubert_norepeating/100000",
]
base_dataset_dir = "/mntcephfs/lab_data/shoinoue/Dataset/"
dataset_paths = [
    "PD-AST/SLT/English",
    "PD-AST/SLT/Hindi",
    "PD-AST/SLT/Korean",
    "PD-AST/ASI/Hindi",
    "PD-AST/TNI/Hindi",
    "PD-AST/HKK/Korean",
    "PD-AST/YDCK/Korean",
    "CMU-ARCTIC/SLT",
    "L2-ARCTIC/ASI",
    "L2-ARCTIC/TNI",
    "L2-ARCTIC/HKK",
    "L2-ARCTIC/YDCK",
]

In [6]:
fs = 16000
save_dir = f"./../../beaqlejs/audio/20240815_spaccent/"
testsamples = list(np.load("data_split_ARCTIC.npy", allow_pickle=True))[2]
for ij in range(2):
    base_dir = [base_model_dir, base_dataset_dir][ij]
    dirnames = [model_paths, dataset_paths][ij]
    for dirname in dirnames:
        print(dirname)
        accentdir = save_dir + f'{"___".join(dirname.split("/"))}'
        os.makedirs(accentdir, exist_ok=True)
        files = glob.glob(base_dir + f"{dirname}/wav/*.wav")
        files.sort()
        for path in tqdm(files):
            basename = os.path.basename(path)[:-4]
            if not(basename in testsamples):
                continue
            save_path = accentdir + f"/{basename}.wav"
            audio, _ = librosa.load(path, sr=fs)
            write_wav(save_path, fs, audio)

VTN_fine-tuning_nocondition_gt2syn_melmel_hubert_norepeating/100000


100%|██████████| 105/105 [00:00<00:00, 264.81it/s]


VTN_fine-tuning_nocondition_mix2synVCTK3hr_melmel_hubert_norepeating/100000


100%|██████████| 100/100 [00:00<00:00, 251.23it/s]


VTN_fine-tuning_nocondition_gt2syn_80mel80mel_hubert_norepeating/100000


...

100%|██████████| 1131/1131 [00:22<00:00, 51.32it/s] 


L2-ARCTIC/HKK


100%|██████████| 1131/1131 [00:26<00:00, 43.39it/s]


L2-ARCTIC/YDCK


100%|██████████| 1131/1131 [00:28<00:00, 39.82it/s]
